# Experiment 04: Amazon Planet

This experiment uses the data from the Kaggle competition [Planet: Understanding the Amazon from Space](https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/leaderboard). Here we use a pretrained ResNet50 model to generate the features from the dataset.

For details of virtual machine we used and the versions of LightGBM and XGBoost, please refer to [experiment 1](01_airline.ipynb).

In [1]:
import sys
from collections import defaultdict
import numpy as np
import pkg_resources
from libs.loaders import load_planet_kaggle
from libs.planet_kaggle import threshold_prediction
from libs.timer import Timer
from libs.utils import get_number_processors
from lightgbm import LGBMClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import json
import warnings; warnings.simplefilter('ignore')

print("System version: {}".format(sys.version))
print("XGBoost version: {}".format(pkg_resources.get_distribution('xgboost').version))
print("LightGBM version: {}".format(pkg_resources.get_distribution('lightgbm').version))

Using TensorFlow backend.


System version: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
XGBoost version: 0.6
LightGBM version: 0.2


In [2]:
%env MOUNT_POINT=/datadrive

env: MOUNT_POINT=/datadrive


The images are loaded and featurised using a pretrained ResNet50 model available from Keras

In [3]:
X_train, y_train, X_test, y_test = load_planet_kaggle()

In [4]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(35000, 2048)
(35000, 17)
(5479, 2048)
(5479, 17)


## XGBoost 



In [5]:
number_processors = get_number_processors()
print("Number of processors: ", number_processors)

Number of processors:  24


We will use a one-v-rest. So each classifier will be responsible for determining whether the assigned tag applies to the image

In [6]:
def train_and_validate_xgboost(params, train_features, train_labels, validation_features, num_boost_round):
    n_classes = train_labels.shape[1]
    y_val_pred = np.zeros((validation_features.shape[0], n_classes))
    time_results = defaultdict(list)
    for class_i in tqdm(range(n_classes)):
        dtrain = xgb.DMatrix(data=train_features, label=train_labels[:, class_i])
        dtest = xgb.DMatrix(data=validation_features)
        with Timer() as t:
            model = xgb.train(params, dtrain, num_boost_round=num_boost_round)
        time_results['train_time'].append(t.interval)
        
        with Timer() as t:
            y_val_pred[:, class_i] = model.predict(dtest)
        time_results['test_time'].append(t.interval)
        
    return y_val_pred, time_results

In [7]:
def train_and_validate_lightgbm(params, train_features, train_labels, validation_features, num_boost_round):
    n_classes = train_labels.shape[1]
    y_val_pred = np.zeros((validation_features.shape[0], n_classes))
    time_results = defaultdict(list)
    for class_i in tqdm(range(n_classes)):
        lgb_train = lgb.Dataset(train_features, train_labels[:, class_i], free_raw_data=False)
        with Timer() as t:
            model = lgb.train(params, lgb_train, num_boost_round = num_boost_round)
        time_results['train_time'].append(t.interval)
        
        with Timer() as t:
            y_val_pred[:, class_i] = model.predict(validation_features)
        time_results['test_time'].append(t.interval)
        
    return y_val_pred, time_results

In [8]:
metrics_dict = {
    'Accuracy': accuracy_score,
    'Precision': lambda y_true, y_pred: precision_score(y_true, y_pred, average='samples'),
    'Recall': lambda y_true, y_pred: recall_score(y_true, y_pred, average='samples'),
    'F1': lambda y_true, y_pred: f1_score(y_true, y_pred, average='samples'),
}

def classification_metrics(metrics, y_true, y_pred):
    return {metric_name:metric(y_true, y_pred) for metric_name, metric in metrics.items()}

In [9]:
results_dict = dict()
num_rounds = 50

Now we are going to define the different models.

In [10]:
xgb_params = {'max_depth':6, 
              'objective':'binary:logistic', 
              'min_child_weight':1, 
              'learning_rate':0.1, 
              'scale_pos_weight':2, 
              'gamma':0.1, 
              'reg_lamda':1, 
              'subsample':1,
              'nthread':number_processors
             }

In [11]:
y_pred, timing_results = train_and_validate_xgboost(xgb_params, X_train, y_train, X_test, num_boost_round=num_rounds)

100%|██████████| 17/17 [05:36<00:00, 19.88s/it]


In [12]:
results_dict['xgb']={
    'train_time': np.sum(timing_results['train_time']),
    'test_time': np.sum(timing_results['test_time']),
    'performance': classification_metrics(metrics_dict, 
                                          y_test, 
                                          threshold_prediction(y_pred, threshold=0.1)) 
}

In [13]:
xgb_hist_params = {'max_depth':0, 
                  'max_leaves':2**6, 
                  'objective':'binary:logistic', 
                  'min_child_weight':1, 
                  'learning_rate':0.1, 
                  'scale_pos_weight':2, 
                  'gamma':0.1, 
                  'reg_lamda':1, 
                  'subsample':1,
                  'nthread':number_processors,
                  'tree_method':'hist', 
                  'grow_policy':'lossguide',
                  'max_bins': 63
                 }

In [14]:
y_pred, timing_results = train_and_validate_xgboost(xgb_hist_params, X_train, y_train, X_test, num_boost_round=num_rounds)

100%|██████████| 17/17 [35:26<00:00, 116.33s/it]


In [15]:
results_dict['xgb_hist']={
    'train_time': np.sum(timing_results['train_time']),
    'test_time': np.sum(timing_results['test_time']),
    'performance': classification_metrics(metrics_dict, 
                                          y_test, 
                                          threshold_prediction(y_pred, threshold=0.1)) 
}

## LightGBM

In [16]:
lgb_params = {'num_leaves': 2**6,
             'learning_rate': 0.1,
             'scale_pos_weight': 2,
             'min_split_gain': 0.1,
             'min_child_weight': 1,
             'reg_lambda': 1,
             'subsample': 1,
             'objective':'binary',
             'task': 'train',
             'nthread':number_processors,
             'max_bin': 63
             }

In [17]:
y_pred, timing_results = train_and_validate_lightgbm(lgb_params, X_train, y_train, X_test, num_boost_round=num_rounds)

100%|██████████| 17/17 [03:13<00:00,  7.91s/it]


In [18]:
results_dict['lgbm']={
    'train_time': np.sum(timing_results['train_time']),
    'test_time': np.sum(timing_results['test_time']),
    'performance': classification_metrics(metrics_dict, 
                                          y_test, 
                                          threshold_prediction(y_pred, threshold=0.1)) 
}

Finally, we show the results.

In [19]:
# Results
print(json.dumps(results_dict, indent=4, sort_keys=True))

{
    "lgbm": {
        "performance": {
            "Accuracy": 0.37233071728417594,
            "F1": 0.822258366139549,
            "Precision": 0.7439077632634851,
            "Recall": 0.9734099462015139
        },
        "test_time": 0.1641630920021271,
        "train_time": 194.57900593099475
    },
    "xgb": {
        "performance": {
            "Accuracy": 0.34057309728052565,
            "F1": 0.8048263053953228,
            "Precision": 0.7184218531362171,
            "Recall": 0.9766441564762427
        },
        "test_time": 0.1852665030019125,
        "train_time": 313.8951129560046
    },
    "xgb_hist": {
        "performance": {
            "Accuracy": 0.37871874429640445,
            "F1": 0.8220252909027159,
            "Precision": 0.7447899193746976,
            "Recall": 0.9720717197264013
        },
        "test_time": 0.19687007299944526,
        "train_time": 2115.2851170680005
    }
}


This dataset shows an interesting behavior. It is the only notebook where XGBoost hist behaves worse than XGBoost. The reason could be because the number of features is high, 2048, and that could be causing a memory overhead. LightGBM and the standard version of XGBoost can manage this high number of features, so there is no overhead. You can try to use a higher complexity to improve the performance. For example, setting `max_depth=8` in XGBoost, `max_leaves=2**8` in XGBoost hist and `num_leaves=2**6` in LightGBM. This will cause an overhead in XGBoost hist.